In [4]:
pip install sec-api

Note: you may need to restart the kernel to use updated packages.


In [12]:
from sec_api import QueryApi
import sec_API_KEY#the .py file that store my api key
api_key = sec_API_KEY.API_KEY
queryApi = QueryApi(api_key)

In [13]:
formType:("10-K", "10-KT", "10KSB", "10KT405", "10KSB40", "10-K405")

In [14]:
#find the most recently filed 10-K filing from Tesla
query = {
  "query": { "query_string": { 
      "query": "formType:\"10-K\" AND ticker:TSLA", # only 10-Ks
  }},
  "from": "0", # start returning matches from position null, i.e. the first matching filing 
  "size": "1"  # return just one filing
}

response = queryApi.get_filings(query) 
#url in linkToFilingDetails

In [15]:
import json 
print(json.dumps(response["filings"][0], indent=2))


{
  "ticker": "TSLA",
  "formType": "10-K",
  "accessionNo": "0001628280-24-002390",
  "cik": "1318605",
  "companyNameLong": "Tesla, Inc. (Filer)",
  "companyName": "Tesla, Inc.",
  "linkToFilingDetails": "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm",
  "description": "Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]",
  "linkToTxt": "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/0001628280-24-002390.txt",
  "filedAt": "2024-01-26T21:00:20-05:00",
  "documentFormatFiles": [
    {
      "sequence": "1",
      "size": "2672746",
      "documentUrl": "https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm",
      "description": "10-K",
      "type": "10-K"
    },
    {
      "sequence": "2",
      "size": "207611",
      "documentUrl": "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-2023x12x31xex211.htm",
      "description": "EX-21.1",
      "typ

In [ ]:
#Query API returns a maximum of 200 filings per search request and a maximum of 10,000 filings per search universe
#iterated through all filings filed between 1997 and 2024

In [2]:
#1. Generate a list of URLs of all 10-K filings
base_query = {
  "query": { 
      "query_string": { 
          "query": "PLACEHOLDER", # this will be set during runtime 
          "time_zone": "America/New_York"
      } 
  },
  "from": "0",
  "size": "200", # don't change this
  # sort returned filings by the filedAt key/value
  "sort": [{ "filedAt": { "order": "desc" } }]
}


In [14]:
# open the file we use to store the filing URLs
log_file = open("filing_urls.txt", "a")

# start with filings filed in 2024, then 2023, 2022, ... up to 1997
# fetch all filings filed from 2022-1997
for year in range(2024, 1996, -1):
  print("Starting download for year {year}".format(year=year))
  
  # a single search universe is represented as a month of the given year
  for month in range(1, 13, 1):
    # get 10-Q and 10-Q/A filings filed in year and month
    # resulting query example: "formType:\"10-Q\" AND filedAt:[2021-01-01 TO 2021-01-31]"
    universe_query = \
        "formType:(\"10-K\", \"10-KT\", \"10KSB\", \"10KT405\", \"10KSB40\", \"10-K405\") AND " + \
        "filedAt:[{year}-{month:02d}-01 TO {year}-{month:02d}-31]" \
        .format(year=year, month=month)
  
    # set new query universe for year-month combination
    base_query["query"]["query_string"]["query"] = universe_query;

    # paginate through results by increasing "from" parameter 
    # until we don't find any matches anymore
    # uncomment next line to fetch all 10,000 filings
    for from_batch in range(0, 9800, 200): #200 is the step value
      # set new "from" starting position of search 
      base_query["from"] = from_batch;

      response = queryApi.get_filings(base_query)

      # no more filings in search universe
      if len(response["filings"]) == 0:
        break;

      # for each filing, only save the URL pointing to the filing itself 
      # and ignore all other data. 
      # the URL is set in the dict key "linkToFilingDetails"
      urls_list = list(map(lambda x: x.get("linkToFilingDetails"), response["filings"]))
      urls_list = [url for url in urls_list if url is not None]
    

      # transform list of URLs into one string by joining all list elements
      # and add a new-line character between each element.
      urls_string = "\n".join(urls_list) + "\n"
      
      log_file.write(urls_string)

    print("Filing URLs downloaded for {year}-{month:02d}".format(year=year, month=month))

log_file.close()

print("All URLs downloaded")

Starting download for year 2024
Filing URLs downloaded for 2024-01
Filing URLs downloaded for 2024-02
Filing URLs downloaded for 2024-03
Filing URLs downloaded for 2024-04
Filing URLs downloaded for 2024-05
Filing URLs downloaded for 2024-06
Filing URLs downloaded for 2024-07
Filing URLs downloaded for 2024-08
Filing URLs downloaded for 2024-09
Filing URLs downloaded for 2024-10
Filing URLs downloaded for 2024-11
Filing URLs downloaded for 2024-12
Starting download for year 2023
Filing URLs downloaded for 2023-01
Filing URLs downloaded for 2023-02
Filing URLs downloaded for 2023-03
Filing URLs downloaded for 2023-04
Filing URLs downloaded for 2023-05
Filing URLs downloaded for 2023-06
Filing URLs downloaded for 2023-07
Filing URLs downloaded for 2023-08
Filing URLs downloaded for 2023-09
Filing URLs downloaded for 2023-10
Filing URLs downloaded for 2023-11
Filing URLs downloaded for 2023-12
Starting download for year 2022
Filing URLs downloaded for 2022-01
Filing URLs downloaded for 20